In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import zarr
import tensorstore as ts
import dask.array as da
import napari

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append('..')
from tracks_interactions.graph.family_graph import FamilyGraphWidget
from tracks_interactions.widget.track_navigation_widget import TrackNavigationWidget
from tracks_interactions.widget.track_modification_widget import TrackModificationWidget

In [2]:
os.environ['NAPARI_PERFMON'] = '0'

from napari.settings import get_settings
settings = get_settings()
settings.experimental.async_
#settings.experimental.async_=True

False

In [3]:
# pathway to labels
#labels_zarr_path = r'D:\kasia\tracking\zarr_tests\labels_double_1024.zarr'
labels_zarr_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\labels_double_flat.zarr'

# open using a tensorstore

# Create the specification for opening the Zarr dataset
spec = {
  'driver': 'zarr',
  'kvstore': {
    'driver': 'file',  # Use 'file' for local filesystem
    'path': labels_zarr_path,
  },
  # Optionally, specify other parameters like 'dtype', 'shape', etc., if needed
}

# Open the dataset asynchronously
dataset_future = ts.open(spec)

# Wait for the dataset to be fully opened (synchronous operation)
dataset = dataset_future.result()

In [4]:
# get images
ch0_path = r'D:\kasia\tracking\E6_exp\E6_C0.zarr'
ch1_path = r'D:\kasia\tracking\E6_exp\E6_C1.zarr'

ch0_list = []
for level in range(1,5):
    ch0_list.append(da.from_zarr(ch0_path,level))

ch1_list = []
for level in range(1,5):
    ch1_list.append(da.from_zarr(ch1_path,level))

In [5]:
# display channels and labels
viewer = napari.Viewer()

ch1 = viewer.add_image(ch0_list, name='ch1', colormap = 'green',blending='additive',contrast_limits=[0, 2048])
ch2 = viewer.add_image(ch1_list, name='ch2', colormap = 'red',blending='additive',contrast_limits=[0, 2048])

labels_layer = viewer.add_labels(dataset,name='Labels')

In [6]:
# get access to the database
new_db_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\Exp6_gardener.db'
#r'D:\kasia\tracking\E6_exp\sample_data\cells_database_expand.db'
engine = create_engine(f'sqlite:///{new_db_path}')
session = sessionmaker(bind=engine)()

### Family graph display

In [7]:
plot_widget = FamilyGraphWidget(viewer,session)
viewer.window.add_dock_widget(plot_widget,area='bottom')

### Navigation widget

In [8]:
track_navigation_widget = TrackNavigationWidget(viewer,session)
viewer.window.add_dock_widget(track_navigation_widget,area='right',name='Track Navigation')

### Modification widget

In [9]:
track_modification_widget = TrackModificationWidget(viewer,session)
viewer.window.add_dock_widget(track_modification_widget,area='right',name='Track Modifications')

### Signal graph

In [10]:
from tracks_interactions.graph.signal_graph import SignalGraph
signal_list = ['ch0_nuc','ch0_cyto']
color_list = ['yellow','green']
graph_widget = SignalGraph(viewer,session,signal_list,color_list)
viewer.window.add_dock_widget(graph_widget,area='right',name='Cell Signals')

### Modify cell

In [12]:
from tracks_interactions.widget.cell_modification_widget import CellModificationWidget
cell_modification_widget = CellModificationWidget(viewer,session,ch_list=[ch0_list[0],ch1_list[0]])
viewer.window.add_dock_widget(cell_modification_widget,area='right',name='Save Cell')